In [49]:
# question: 我手動用我給你的pGlyco篩分數總共有153筆，expand後應有152*2=306，但我把light orange+light blue只有278筆；Byonic手動497筆，green+light orange只有491，中間不知道有甚麼Bug? by 瑜君學姊

# DEBUG EXAMPLE: 2 IN 1 OUT (FCWF CELL-EXPRESSED UU4)
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib import ticker
from IPython.display import display, HTML
import re # finding specific patterns in str
import textwrap # split text into equal parts
import collections # return repeated items in list
from collections import OrderedDict
import time
import sys

start_time = time.time()

# read in pglyco as df
pglycofile = 'pglyco_Fcwf_T_Chy_Mammalian'
pglyco_df = pd.read_excel('%s.xlsx'%pglycofile, header = 0)
pglyco_df = pglyco_df.fillna('N/A')
pglyco_df = pglyco_df.sort_values(by=['Scan'])
pglyco_df = pglyco_df.reset_index(drop = True)
# display(HTML(pglyco_df.to_html()))

## preprocess the pglycofile first
# output column name
print('original pglyco columns:\n%s\n'%pglyco_df.columns)

# record original data size
print('original pglyco data size:\nrow: %s\ncol: %s\n'%(pglyco_df.shape[0], pglyco_df.shape[1]))

# check how many rows can pass pepscore > 5 & glyscore > 4
p_pass_before = pglyco_df[(pglyco_df['PepScore'] > 5) & (pglyco_df['GlyScore'] > 4)]
print('pglyco data size meeting threshold:\nrow: %s\ncol: %s\n'%(p_pass_before.shape[0], p_pass_before.shape[1]))
# change J back to N as column named peptide(J-->N)
pglyco_df['Peptide'] = pglyco_df['Peptide'].str.replace('J','N')

# analyze sequon in pglyco file
pglyco_sequon = pglyco_df['Peptide'].str.findall('(N[ARNDBCEQZGHILKMFSTWYV]T)|(N[ARNDBCEQZGHILKMFSTWYV]S)').tolist()
pglyco_sequon_lst = []
for t in pglyco_sequon:
    t = str(t)
    res = re.findall('[ARNDBCEQZGHILKMFSTWYVP]', t)
    res = ''.join(res)
    res = textwrap.wrap(res, 3)
    if res == []:
        pglyco_sequon_lst.append('N/A')
    elif len(res) == 1:
        pglyco_sequon_lst.append(res[0])
    else:
        pglyco_sequon_lst.append(res)
        
# print('pglyco_sequon: %s'%pglyco_sequon_lst)
pglyco_df.insert(pglyco_df.columns.get_loc('Peptide') + 1 , 'Sequon', pglyco_sequon_lst , True)
# display(HTML(pglyco_df.to_html()))

# replace H, N, A, F, G symbols w/ Hex, HexNAc, NeuAc, Fuc, NeuGc
byonicstyle = []
if 'Glycan(H,N,A,G,F)' in pglyco_df.columns:
    glycan_num = pglyco_df['Glycan(H,N,A,G,F)'].tolist()
    for i in glycan_num:
        i = i.split(' ')
#         print(i)
        new_order = [1, 0, 4, 2, 3]
        i = [i[x] for x in new_order]
#         print('new: %s'%i)
        if i[0] != '0':
            n = 'HexNAc(%s)'%(i[0])
        else :
            n = ''
        if i[1] != '0':
            h = 'Hex(%s)'%(i[1])
        else:
            h = ''
        if i[2] != '0':
            f = 'Fuc(%s)'%(i[2])
        else:
            f = ''
        if i[3] != '0':
            a = 'NeuAc(%s)'%(i[3])
        else:
            a = ''
        if i[4] != '0':
            g = 'NeuGc(%s)'%(i[4])
        else:
            g = ''
        each_byonicstyle = n + h + f + a + g
        byonicstyle.append(each_byonicstyle)
elif 'Glycan(H,N,A,F)' in pglyco_df.columns:
    glycan_num = pglyco_df['Glycan(H,N,A,F)'].tolist()
    for i in glycan_num:
        i = i.split(' ')
        new_order = [1, 0, 3, 2]
        i = [i[x] for x in new_order]
        if i[0] != '0':
            n = 'HexNAc(%s)'%(i[0])
        else :
            n = ''
        if i[1] != '0':
            h = 'Hex(%s)'%(i[1])
        else:
            h = ''
        if i[2] != '0':
            f = 'Fuc(%s)'%(i[2])
        else:
            f = ''
        if i[3] != '0':
            a = 'NeuAc(%s)'%(i[3])
        else:
            a = ''
        each_byonicstyle = n + h + f + a
        byonicstyle.append(each_byonicstyle)
else:
    sys.exit('Please check if there are glycans other than H, N, A, G, F or if column name has changed. This app will stop.')
# print(byonicstyle)
pglyco_df.insert(pglyco_df.columns.get_loc('GlycanComposition') + 1 , 'GlycanComposition_ByonicStyle', byonicstyle , True)
# display(HTML(pglyco_df.to_html()))

# if the etdscan is not blank, duplicate the row and change the duplicated scan to etdscan (micmic byonic format)
pglyco_df.insert(pglyco_df.columns.get_loc('Scan') + 1 , 'FragmentType', 'hcd' , True) # insert 'fragment type' col
row_to_duplicate = pglyco_df[pglyco_df['ETDScan'] != 'N/A'].copy() # assume the missing ETDScan in pglyco is represented as blank & later filled w/ N/A
row_to_duplicate['FragmentType'] = 'ethcd'
row_to_duplicate['Scan'] = row_to_duplicate['ETDScan']
pglyco_df = pd.concat([pglyco_df, row_to_duplicate]) # duplicate w/ index
pglyco_df = pglyco_df.sort_values(by = ['Scan']) # sort by scan directly, the duplicated ethcd rows can be separated from hcd rows
# check data size after etdscan expansion
print('pglyco data size after etdscan expansion:\nrow: %s\ncol: %s\n'%(pglyco_df.shape[0], pglyco_df.shape[1]))
p_pass_after = pglyco_df[(pglyco_df['PepScore'] > 5) & (pglyco_df['GlyScore'] > 4)]
print('pglyco data size meeting threshold after etdscan expansion:\nrow: %s\ncol: %s\n'%(p_pass_after.shape[0], p_pass_after.shape[1]))
# display(HTML(pglyco_df.to_html()))
print('----- pGlyco data preprocessing completed. -----\n')

## import byonic raw excel file (contain all the info.) to compare
byonicfile = 'byonic_Fcwf_T_Chy_Mammalian'
byonic_df = pd.read_excel('%s.xlsx'%byonicfile, header = 0)
byonic_df = byonic_df.fillna('N/A')
print('original byonic columns:\n%s\n'%byonic_df.columns)
# record original data size
print('original byonic data size:\nrow: %s\ncol: %s\n'%(byonic_df.shape[0], byonic_df.shape[1]))
# check how many rows can pass pepscore > 5 & glyscore > 4
b_pass_before = byonic_df[(byonic_df['Score'] > 200) & (byonic_df['PEP_x000D_\n2D'] < 0.001)]
print('byonic data size meeting threshold:\nrow: %s\ncol: %s\n'%(b_pass_before.shape[0], b_pass_before.shape[1]))
# extract 'scan' from 'scan #' in byonic file & add a 'Scan' column
byonic_scan = byonic_df['Scan #'].tolist()
byonic_scan_lst = []
for scan in byonic_scan:
    scan = scan.split(' ')[-1].split('=')[-1]
    scan = int(scan)
#     print(scan)
    byonic_scan_lst.append(scan)
byonic_df.insert(byonic_df.columns.get_loc('Scan_x000D_\nTime') + 1 , 'Scan', byonic_scan_lst , True)
byonic_df = byonic_df.sort_values(by = ['Scan'])
byonic_df = byonic_df.reset_index(drop = True)
# add 'PureSequence' column to byonic file
if 'Sequence\r\n(unformatted)' in byonic_df.columns: # deal w/ dif byonic version
    byonic_seq = byonic_df['Sequence\r\n(unformatted)'].str.findall('[ARNDBCEQZGHILKMFSTWYVP]').tolist()
    byonic_seq = [''.join(each_pure) for each_pure in byonic_seq]
    byonic_df.insert(byonic_df.columns.get_loc('Sequence\r\n(unformatted)') + 1 , 'PureSequence', byonic_seq , True)
elif 'Sequence' in byonic_df.columns:
    byonic_seq = byonic_df['Sequence'].str.findall('[ARNDBCEQZGHILKMFSTWYVP]').tolist()  
    byonic_seq = [''.join(each_pure) for each_pure in byonic_seq]
    byonic_df.insert(byonic_df.columns.get_loc('Sequence') + 1 , 'PureSequence', byonic_seq , True)
else:
    sys.exit('Please check if sequence column name has changed. This app will stop.')
    
# add 'Sequon' column to byonic file
byonic_sequon_lst = []
byonic_sequon = byonic_df['PureSequence'].str.findall('(N[ARNDBCEQZGHILKMFSTWYV]T)|(N[ARNDBCEQZGHILKMFSTWYV]S)').tolist()
for t in byonic_sequon:
#     print(t)
    t = str(t)
    res = re.findall('[ARNDBCEQZGHILKMFSTWYVP]', t)
    res = ''.join(res)
    res = textwrap.wrap(res, 3)
    if res == []:
        byonic_sequon_lst.append('N/A')
    elif len(res) == 1:
        byonic_sequon_lst.append(res[0])
    else:
        byonic_sequon_lst.append(res)
byonic_df.insert(byonic_df.columns.get_loc('PureSequence') + 1 , 'Sequon', byonic_sequon_lst , True)
# add N/A col 'Pair' for later modification (pair: cal.m/z same, cal.m same, pureseq same, scan difference <= 5, hcd before ethcd)
byonic_df.insert(byonic_df.columns.get_loc('Scan_x000D_\nTime') + 1 , 'Pair', 'N/A' , True)
# potential_pair = byonic_df[['Calc.\r\nm/z', 'Calc.\r\nMH', 'PureSequence']][byonic_df.duplicated(subset=['Calc.\r\nm/z', 'Calc.\r\nMH', 'PureSequence'], keep=False)]
potential_pair = byonic_df[byonic_df.duplicated(subset=['Calc._x000D_\nm/z', 'Calc.M', 'PureSequence'], keep=False)].sort_values(['Calc._x000D_\nm/z'])
# print(potential_pair)
mz_gp = sorted(list(set(potential_pair['Calc._x000D_\nm/z'].tolist())))
pair_cnt = 0
all_hcd_ind = []
all_etd_ind = []
# mh_gp = sorted(list(set(potential_pair['Calc.\r\nMH'].tolist())))
# pureseq_gp = sorted(list(set(potential_pair['PureSequence'].tolist())))
# gp_cnt = 0
for i in mz_gp:
    gp = potential_pair[potential_pair['Calc._x000D_\nm/z'] == i][potential_pair.duplicated(subset=['Calc.M', 'PureSequence'], keep=False)]
    gp = gp.sort_values(['Scan'])
#     print('gp:\n%s\n'%gp[['Scan', 'Fragment\r\nType']])
    # skip the gp w/o ethcd & find pairs: last criterion -> scan dif <= 5
    if 'hcd' in gp['Fragment_x000D_\nType'].tolist() and 'ethcd' in gp['Fragment_x000D_\nType'].tolist():
        # from hcd find the nearest ethcd below
        pair_candidate = gp[(gp['Fragment_x000D_\nType'] == 'hcd') & (gp['Fragment_x000D_\nType'].shift(-1) == 'ethcd') & (gp['Scan'].shift(-1) - gp['Scan'] <= 5)]
#         print('pair_candidate:\n%s\n'%pair_candidate[['Scan', 'Fragment\r\nType']])
        hcd_iloc = [gp.index.get_loc(ind) for ind in pair_candidate.index] # get the positions of the hcd in pairs in gp df
# #         print('hcd_ind:\n%s\n'%hcd_loc)
#         pair_lst = ['pair%s'%(i+1+pair_cnt) for i in range(len(hcd_iloc))]
# #         print('pair_lst:%s'%pair_lst)
#         gp['Pair'].iloc[hcd_iloc] = pair_lst
        etd_iloc = list(np.array(hcd_iloc) + 1)
#         gp['Pair'].iloc[etd_iloc] = pair_lst
#         pair_cnt += len(pair_lst) 
# #         print('NEW_gp:\n%s\n'%gp[['Scan', 'Fragment\r\nType', 'Pair']])
#         byonic_df.loc[gp.index.tolist()] = gp
        hcd_ind = pair_candidate.index.tolist()
#         print('hcd_ind:\n%s\n'%hcd_ind)
        etd_ind = [gp.index[i] for i in etd_iloc] 
#         print('etd_ind:\n%s\n'%etd_ind)
        all_hcd_ind.extend(hcd_ind)
        all_etd_ind.extend(etd_ind)
    else:
        pass
all_hcd_ind.sort()
all_etd_ind.sort()
# print('all_hcd_ind:\n%s\n'%all_hcd_ind)
# print('all_etd_ind:\n%s\n'%all_etd_ind)
byonic_df['Pair'].iloc[all_hcd_ind] = ['pair%s'%(i+1) for i in range(len(all_hcd_ind))]
byonic_df['Pair'].iloc[all_etd_ind] = ['pair%s'%(i+1) for i in range(len(all_etd_ind))]
# display(HTML(byonic_df.to_html()))
print('byonic data size after preprocessing:\nrow: %s\ncol: %s\n'%(byonic_df.shape[0], byonic_df.shape[1]))
b_pass_after = byonic_df[(byonic_df['Score'] > 200) & (byonic_df['PEP_x000D_\n2D'] < 0.001)]
print('byonic data size meeting threshold after preprocessing:\nrow: %s\ncol: %s\n'%(b_pass_after.shape[0], b_pass_after.shape[1]))
print('----- Byonic data preprocessing completed. -----\n')

# read in byos file
byosfile = 'UU4__Fcwf'
byos_df = pd.read_excel('%s.xlsx'%byosfile, header = 0)
# output column name
print('original byos columns:\n%s\n'%byos_df.columns)
# extract needed columns: Scan Number(s)(Posit), MS Alias name, XIC area summed, XIC AUC, Apex Int.(Posit), Calc.M, Sequence
byos_df = byos_df[['Scan Number(s)_x000D_\n(Posit)', 'MS2 Search_x000D_\nAlias name', 'XIC area_x000D_\nsummed', 'XIC_x000D_\nAUC', 'Apex Int._x000D_\n(Posit)', 'Calc.M', 'Sequence']]
byos_df = byos_df.fillna('N/A')
# record extracted data size
print('extracted byos data size:\nrow: %s\ncol: %s\n'%(byos_df.shape[0], byos_df.shape[1]))
# output extracted column name
print('extracted byos columns:\n%s\n'%byos_df.columns)
# drop the row w/ multiple scan numbers (data type would be str)
# print('before:\n%s\n'%len(byos_df))  
byos_df = byos_df[byos_df['Scan Number(s)_x000D_\n(Posit)'].apply(lambda x: isinstance(x, int))]
# print('after:\n%s\n'%len(byos_df)) 
# sort byos by scan
byos_df = byos_df.sort_values(by=['Scan Number(s)_x000D_\n(Posit)'])
byos_df = byos_df.reset_index(drop = True)
# add 'PureSequence' col
byos_df['Sequence'] = byos_df['Sequence'].str.upper()
byos_df['PureSequence'] = byos_df['Sequence'].str.replace('.','')
# display(HTML(byos_df.to_html()))
print('byos data size after preprocessing:\nrow: %s\ncol: %s\n'%(byos_df.shape[0], byos_df.shape[1]))
print('----- Byos data preprocessing completed. -----\n')

## combine scans from the two files
print('----- Start combining pGlyco & Byonic & Byos. -----\n')
# combined data based on 'Scan' in byonic & pglyco
byonic_scanasid = byonic_df.copy()
new_byonic_col = [n + '[Byonic]' if n != 'Scan' else n for n in byonic_scanasid.columns]
# print('\nnew_byonic_col:\n%s\n'%new_byonic_col)
byonic_scanasid.columns = new_byonic_col
pglyco_scanasid = pglyco_df.copy()
new_pglyco_col = [n + '[pGlyco]' if n != 'Scan' else n for n in pglyco_scanasid.columns]
pglyco_scanasid.columns = new_pglyco_col
# print('new_pglyco_col:\n%s\n'%new_pglyco_col)
byos_scanasid = byos_df.copy()
new_byos_col = [n + '[Byos]' if n != 'Scan Number(s)_x000D_\n(Posit)' else n for n in byos_scanasid.columns]
byos_scanasid.columns = new_byos_col
# print('new_byos_col:\n%s\n'%new_byos_col)
byonic_scanasid = byonic_scanasid.set_index('Scan')
pglyco_scanasid = pglyco_scanasid.set_index('Scan')
byos_scanasid = byos_scanasid.set_index('Scan Number(s)_x000D_\n(Posit)')
# display(HTML(byos_scanasid.to_html()))
# align scan & concat (all align on row to make row number all the same)
a1, a2 = byonic_scanasid.align(pglyco_scanasid, join = 'outer', axis = 0) # row: a1 = a2
a1, a3 = a1.align(byos_scanasid, join = 'outer', axis = 0) # row: a1 = a2 = a3
a2, a3 = a2.align(byos_scanasid, join = 'outer', axis = 0) # row: a2 = a1 = a3
all_combined_df = pd.concat([a1,a2,a3], axis = 1)
all_combined_df.index.name = 'Scan'
all_combined_df.reset_index(level=0, inplace=True)
# change all nan to blank -1
all_combined_df = all_combined_df.fillna(-1)
# all_combined_df.to_excel('test.xlsx', index = False)  

## result post-processing
# glycan comprison: only present in byonic -> b, only present in pglyco -> p, both the same -> b+p, not the same -> b/p
conditions = [
    (all_combined_df['Glycans[Byonic]'] != -1) & (all_combined_df['GlycanComposition_ByonicStyle[pGlyco]'] == -1),
    (all_combined_df['Glycans[Byonic]'] == -1) & (all_combined_df['GlycanComposition_ByonicStyle[pGlyco]'] != -1),
    (all_combined_df['Glycans[Byonic]'] != -1) & (all_combined_df['GlycanComposition_ByonicStyle[pGlyco]'] != -1) & (all_combined_df['Glycans[Byonic]'] == all_combined_df['GlycanComposition_ByonicStyle[pGlyco]']),
    (all_combined_df['Glycans[Byonic]'] != -1) & (all_combined_df['GlycanComposition_ByonicStyle[pGlyco]'] != -1) & (all_combined_df['Glycans[Byonic]'] != all_combined_df['GlycanComposition_ByonicStyle[pGlyco]'])]
choices = ['B', 'P', 'B+P', 'B/P'] 
glycan_source = np.select(conditions, choices) 
all_combined_df.insert(all_combined_df.columns.get_loc('Scan') + 1 , 'GlycanSource', glycan_source , True)

print('\ncombined data shape:\nrow --> %s, column --> %s'%(all_combined_df.shape[0], all_combined_df.shape[1]))

## style apply for excel export
# color the rows below the threshold (threshold [byonic: score > 200 & pep2d < 0.001; pglyco: PepScore>5 & GlyScore>4])

# color the background to separate byonic data from pglyco data
# color code => #ffedcc -> light orange for byonic; #add8e6 -> light blue for pglyco; #FFB6C1 -> light pink for byos w/ dif calm ^ seq from byonic; #FF1493 -> deep pink for byos w/ dif calm & seq from byonic; #FFFF00 -> yellow for byos & byonic all the same

# def bg_color(x):
#     c1 = 'background-color: #98FB98' # light green
#     c2 = 'background-color: #add8e6' # light blue
#     c3 = 'background-color: #ffedcc' # light orange
#     c4 = 'background-color: #FFB6C1' # light pink
#     c5 = 'background-color: #FF1493' # deep pink
#     c6 = 'background-color: #FFFF00' # yellow
#     c = '' 
#     # compare columns
#     # comparison between byonic & pglyco
#     b_mask = (all_combined_df['Score[Byonic]'] > 200) & (all_combined_df['PEP_x000D_\n2D[Byonic]'].abs() < 0.001) & (all_combined_df['PepScore[pGlyco]'] <= 5) & (all_combined_df['GlyScore[pGlyco]'] <= 4)
#     p_mask = (all_combined_df['Score[Byonic]'] <= 200) & (all_combined_df['PEP_x000D_\n2D[Byonic]'].abs() >= 0.001) & (all_combined_df['PepScore[pGlyco]'] > 5) & (all_combined_df['GlyScore[pGlyco]'] > 4)    
#     both_mask = (all_combined_df['Score[Byonic]'] > 200) & (all_combined_df['PEP_x000D_\n2D[Byonic]'].abs() < 0.001) & (all_combined_df['PepScore[pGlyco]'] > 5) & (all_combined_df['GlyScore[pGlyco]'] > 4)
#     bp_range = byonic_scanasid.columns.tolist() + pglyco_scanasid.columns.tolist()
#     # comparison between byonic & byos
#     byos_exclusiveOr_mask = (all_combined_df['Calc.M[Byos]'] != -1) & (all_combined_df['Calc.M[Byonic]'] != -1) & ((all_combined_df['Calc.M[Byos]'] != all_combined_df['Calc.M[Byonic]'])^(all_combined_df['PureSequence[Byos]'] != all_combined_df['PureSequence[Byonic]']))
#     byos_and_mask = (all_combined_df['Calc.M[Byos]'] != -1) & (all_combined_df['Calc.M[Byonic]'] != -1) & (all_combined_df['Calc.M[Byos]'] != all_combined_df['Calc.M[Byonic]']) & (all_combined_df['PureSequence[Byos]'] != all_combined_df['PureSequence[Byonic]'])
#     byos_bothsame_mask = (all_combined_df['Calc.M[Byos]'] != -1) & (all_combined_df['Calc.M[Byonic]'] != -1) & (all_combined_df['Calc.M[Byos]'] == all_combined_df['Calc.M[Byonic]']) & (all_combined_df['PureSequence[Byos]'] == all_combined_df['PureSequence[Byonic]'])
#     bb_range = byos_scanasid.columns.tolist()
#     #DataFrame with same index and columns names as original filled empty strings
#     df1 =  pd.DataFrame(c, index=x.index, columns=x.columns)
#     #modify values of df1 column by boolean mask
#     df1.loc[b_mask, bp_range] = c1
#     print('%s rows will be colored light green.'%len(df1.loc[b_mask, bp_range]))
#     df1.loc[p_mask, bp_range] = c2
#     print('%s rows will be colored light blue.'%len(df1.loc[p_mask, bp_range]))
#     df1.loc[both_mask, bp_range] = c3
#     print('%s rows will be colored light orange.'%len(df1.loc[both_mask, bp_range]))
#     df1.loc[byos_exclusiveOr_mask, bb_range] = c4
#     df1.loc[byos_and_mask, bb_range] = c5
#     df1.loc[byos_bothsame_mask, bb_range] = c6
#     return df1

# print('----- Exporting file... -----\n')
# all_combined_df.style.apply(bg_color, axis=None).to_excel('20210603_debug.xlsx', index = False, engine='openpyxl')  
# print("\nTask completed.\nExecution time: %.2f seconds"%(time.time() - start_time))

# DEBUG
print('\n------- DEBUG -------\n')
# original byonic pass: 497 rows; pglyco pass: 304 rows (after expansion)
b_mask = (all_combined_df['Score[Byonic]'] > 200) & (all_combined_df['PEP_x000D_\n2D[Byonic]'].abs() < 0.001) & ((all_combined_df['PepScore[pGlyco]'] <= 5) | (all_combined_df['GlyScore[pGlyco]'] <= 4))
p_mask = ((all_combined_df['Score[Byonic]'] <= 200) | (all_combined_df['PEP_x000D_\n2D[Byonic]'].abs() >= 0.001)) & (all_combined_df['PepScore[pGlyco]'] > 5) & (all_combined_df['GlyScore[pGlyco]'] > 4)    
both_mask = (all_combined_df['Score[Byonic]'] > 200) & (all_combined_df['PEP_x000D_\n2D[Byonic]'].abs() < 0.001) & (all_combined_df['PepScore[pGlyco]'] > 5) & (all_combined_df['GlyScore[pGlyco]'] > 4)
bp_range = byonic_scanasid.columns.tolist() + pglyco_scanasid.columns.tolist()
bp_range.append('Scan')
# colors
light_g = all_combined_df.loc[b_mask, bp_range] 
light_b = all_combined_df.loc[p_mask, bp_range]
light_o = all_combined_df.loc[both_mask, bp_range]
combined_byonic_pass = pd.concat([light_g, light_o], ignore_index=True) 
combined_byonic_pass = combined_byonic_pass[['Scan', 'Score[Byonic]', 'PEP_x000D_\n2D[Byonic]']].sort_values(['Scan'])
combined_byonic_pass = combined_byonic_pass.rename(columns={'Score[Byonic]': 'Score', 'PEP_x000D_\n2D[Byonic]': 'PEP_x000D_\n2D'})
combined_byonic_pass = combined_byonic_pass.reset_index(drop = True)
print('combined_byonic_pass:\n%s\n'%combined_byonic_pass)
combined_byonic_pass.to_excel('combined_byonic_pass.xlsx', index = False)
combined_pglyco_pass = pd.concat([light_b, light_o], ignore_index=True) 
combined_pglyco_pass = combined_pglyco_pass[['Scan', 'PepScore[pGlyco]', 'GlyScore[pGlyco]']].sort_values(['Scan'])
combined_pglyco_pass = combined_pglyco_pass.rename(columns={'PepScore[pGlyco]': 'PepScore', 'GlyScore[pGlyco]': 'GlyScore'})
combined_pglyco_pass = combined_pglyco_pass.reset_index(drop = True)
print('combined_pglyco_pass:\n%s\n'%combined_pglyco_pass)
combined_pglyco_pass.to_excel('combined_pglyco_pass.xlsx', index = False)
# find the dif between original pass & combined pass
b_pass_after = b_pass_after[['Scan','Score', 'PEP_x000D_\n2D']].sort_values(['Scan'])
b_pass_after = b_pass_after.reset_index(drop = True)
p_pass_after = p_pass_after[['Scan','PepScore', 'GlyScore']].sort_values(['Scan'])
p_pass_after = p_pass_after.reset_index(drop = True)
print('b_pass_after:\n%s\n'%b_pass_after)
b_pass_after.to_excel('b_pass_after.xlsx', index = False)
print('p_pass_after:\n%s\n'%p_pass_after)
p_pass_after.to_excel('p_pass_after.xlsx', index = False)
# byonic_dif = b_pass_after[~b_pass_after.apply(tuple,1).isin(combined_byonic_pass.apply(tuple,1))]
# pglyco_dif = p_pass_after[~p_pass_after.apply(tuple,1).isin(combined_pglyco_pass.apply(tuple,1))]
byonic_dif = combined_byonic_pass[~combined_byonic_pass.apply(tuple,1).isin(b_pass_after.apply(tuple,1))]
pglyco_dif = combined_pglyco_pass[~combined_pglyco_pass.apply(tuple,1).isin(p_pass_after.apply(tuple,1))]
print('byonic_dif:\n%s\n'%byonic_dif)
print('pglyco_dif:\n%s\n'%pglyco_dif)
print(combined_byonic_pass.equals(b_pass_after))
print(combined_pglyco_pass.equals(p_pass_after))

original pglyco columns:
Index(['GlySpec', 'PepSpec', 'RawName', 'Scan', 'RT', 'PrecursorMH',
       'PrecursorMZ', 'Charge', 'Rank', 'Peptide', 'Mod', 'PeptideMH',
       'Glycan(H,N,A,G,F)', 'GlycanComposition', 'PlausibleStruct', 'GlyID',
       'GlyFrag', 'GlyMass', 'GlySite', 'TotalScore', 'PepScore', 'GlyScore',
       'CoreMatched', 'MassDeviation', 'PPM', 'GlyIonRatio', 'byIonRatio',
       'czIonRatio', 'GlyDecoy', 'PepDecoy', 'IsSmallGlycan', 'GlycanPEP',
       'GlycanFDR', 'PeptidePEP', 'PeptideFDR', 'TotalFDR', 'Proteins',
       'Genes', 'ProSites', 'MonoArea', 'IsotopeArea', 'ETDScan',
       'LocalizedSiteGroups', 'LocalizedScore', 'LocalizedIonRatio',
       'PreLocalizedScore'],
      dtype='object')

original pglyco data size:
row: 205
col: 46

pglyco data size meeting threshold:
row: 152
col: 46

pglyco data size after etdscan expansion:
row: 410
col: 49

pglyco data size meeting threshold after etdscan expansion:
row: 304
col: 49

----- pGlyco data preprocessing co

<ipython-input-49-411af7097c60>:198: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gp = potential_pair[potential_pair['Calc._x000D_\nm/z'] == i][potential_pair.duplicated(subset=['Calc.M', 'PureSequence'], keep=False)]
<ipython-input-49-411af7097c60>:198: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gp = potential_pair[potential_pair['Calc._x000D_\nm/z'] == i][potential_pair.duplicated(subset=['Calc.M', 'PureSequence'], keep=False)]
<ipython-input-49-411af7097c60>:198: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gp = potential_pair[potential_pair['Calc._x000D_\nm/z'] == i][potential_pair.duplicated(subset=['Calc.M', 'PureSequence'], keep=False)]
<ipython-input-49-411af7097c60>:198: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gp = potential_pair[potential_pair['Calc._x000D_\nm/z'] == i][potential_pair.duplicated(subset=['Calc.M', 'PureSequence'], keep=Fals

byonic data size after preprocessing:
row: 8060
col: 45

byonic data size meeting threshold after preprocessing:
row: 497
col: 45

----- Byonic data preprocessing completed. -----

original byos columns:
Index(['Row#', 'PID', 'Protein_x000D_\nname', 'Sequence_x000D_\n(unformatted)',
       'Scan Number(s)_x000D_\n(Posit)', 'Sequence', 'XIC area_x000D_\nsummed',
       'XIC_x000D_\nAUC', 'XIC_x000D_\nRatio%', 'Apex Int._x000D_\n(Posit)',
       '_grp_x000D_\nnum', 'Feature_x000D_\nnumber', 'iso',
       'Sample-charge_x000D_\nID', 'Apex Time_x000D_\n(Posit)',
       'MS_x000D_\nAlias name', 'MS2 Search_x000D_\nAlias name', 'Score',
       'PEP_x000D_\n2D', 'Glycans', 'z', 'Calc._x000D_\nm/z', 'Calc.M',
       'Scan Time(s)_x000D_\n(Posit)', 'Fragment_x000D_\ntype(s)', 'Validate',
       'Comment', 'Labels', 'Mod._x000D_\nNames', '_mod_x000D_\nids',
       'Mod._x000D_\nSummary', 'Mod._x000D_\nAAs', 'Var. Pos._x000D_\nProtein',
       'Var. Pos._x000D_\nPeptide', 'Start_x000D_\nAA', 'End

<ipython-input-49-411af7097c60>:257: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  byos_df['PureSequence'] = byos_df['Sequence'].str.replace('.','')


combined_pglyco_pass:
      Scan  PepScore  GlyScore
0     1955  14.11200  79.12835
1     1956  14.11200  79.12835
2     1962  13.67942  90.71321
3     1963  13.67942  90.71321
4     2024  17.20835  85.25100
..     ...       ...       ...
299  17235  56.92902  11.75279
300  17348  11.32334   5.55692
301  17351  11.32334   5.55692
302  17513   7.38356   6.86348
303  17515   7.38356   6.86348

[304 rows x 3 columns]

b_pass_after:
      Scan   Score  PEP_x000D_\n2D
0     1083  1005.4    6.600000e-15
1     1273   952.8    6.600000e-14
2     1340   576.1    1.600000e-06
3     1385   543.7    3.200000e-06
4     1469   395.0    7.900000e-05
..     ...     ...             ...
492  25159   522.0    5.300000e-05
493  25661   539.9    2.900000e-06
494  25671   368.1    4.400000e-05
495  25715   549.7    5.700000e-06
496  25914   343.5    2.900000e-04

[497 rows x 3 columns]

p_pass_after:
      Scan  PepScore  GlyScore
0     1955  14.11200  79.12835
1     1956  14.11200  79.12835
2     1962  13.